In [2]:
!pip install imblearn

  Using cached imbalanced_learn-0.13.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached sklearn_compat-0.1.3-py3-none-any.whl.metadata (18 kB)
  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
Using cached imbalanced_learn-0.13.0-py3-none-any.whl (238 kB)
Using cached sklearn_compat-0.1.3-py3-none-any.whl (18 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0


  You can safely remove it manually.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Enhanced Fraud Detection System - Complete ML Pipeline
# Author: Data Science Team
# Objective: Build a robust fraud detection system with high precision and recall
# Version: 2.0 - Enhanced with better accuracy and fixed encoding issues

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, 
                           roc_auc_score, roc_curve, precision_recall_curve,
                           f1_score, precision_score, recall_score, average_precision_score)

# Advanced ML
try:
    import xgboost as xgb
    HAS_XGBOOST = True
except ImportError:
    print("⚠️ XGBoost not available. Skipping XGBoost model.")
    HAS_XGBOOST = False

try:
    from catboost import CatBoostClassifier
    HAS_CATBOOST = True
except ImportError:
    print("⚠️ CatBoost not available. Skipping CatBoost model.")
    HAS_CATBOOST = False

# Imbalanced Learning
try:
    from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
    from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours
    from imblearn.combine import SMOTETomek
    from imblearn.pipeline import Pipeline as ImbPipeline
    HAS_IMBLEARN = True
except ImportError:
    print("⚠️ imbalanced-learn not available. Using alternative class balancing methods.")
    HAS_IMBLEARN = False

# Utilities
import joblib
from datetime import datetime
import os
from collections import Counter

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("🚀 Enhanced Fraud Detection System Initialized")
print("=" * 60)

# ============================================================================
# 1. ENHANCED DATA SIMULATION WITH REALISTIC PATTERNS
# ============================================================================

def create_enhanced_sample_data(n_samples=10000):
    """Create enhanced sample data with more realistic fraud patterns"""
    
    np.random.seed(42)
    
    print(f"📊 Generating {n_samples:,} sample transactions...")
    
    # Generate base features
    data = {
        'Transaction_ID': [f'TXN_{i:06d}' for i in range(n_samples)],
        'User_ID': [f'USER_{np.random.randint(1000, 9999)}' for _ in range(n_samples)],
    }
    
    # Generate correlated features for more realistic data
    # Risk score influences many other features
    risk_scores = np.random.beta(2, 5, n_samples)  # Skewed towards lower values
    data['Risk_Score'] = risk_scores
    
    # Transaction amount - higher amounts more likely with high risk
    base_amounts = np.random.lognormal(3.0, 1.5, n_samples)
    risk_multiplier = 1 + 2 * risk_scores  # High risk = higher amounts
    data['Transaction_Amount'] = np.clip(base_amounts * risk_multiplier, 1, 10000)
    
    # Account balance - inversely related to risk sometimes
    data['Account_Balance'] = np.random.lognormal(8, 1.2, n_samples)
    data['Account_Balance'] = np.clip(data['Account_Balance'], 500, 100000)
    
    # Transaction types with different fraud propensities
    transaction_types = ['POS', 'Bank Transfer', 'Online', 'ATM Withdrawal']
    type_probs = [0.4, 0.25, 0.25, 0.1]  # Online and transfers riskier
    data['Transaction_Type'] = np.random.choice(transaction_types, n_samples, p=type_probs)
    
    # Device types
    device_types = ['Mobile', 'Laptop', 'Tablet']
    data['Device_Type'] = np.random.choice(device_types, n_samples, p=[0.6, 0.3, 0.1])
    
    # Locations with varying risk levels
    locations = ['New York', 'London', 'Sydney', 'Mumbai', 'Tokyo', 'Lagos', 'Moscow']
    data['Location'] = np.random.choice(locations, n_samples)
    
    # Merchant categories
    merchant_cats = ['Grocery', 'Gas', 'Restaurants', 'Clothing', 'Electronics', 'Travel', 'Entertainment']
    data['Merchant_Category'] = np.random.choice(merchant_cats, n_samples)
    
    # IP flag - correlated with risk score
    ip_flag_prob = 0.02 + 0.15 * risk_scores
    data['IP_Address_Flag'] = np.random.binomial(1, ip_flag_prob, n_samples)
    
    # Previous fraudulent activity
    prev_fraud_prob = 0.05 + 0.2 * risk_scores
    data['Previous_Fraudulent_Activity'] = np.random.binomial(1, prev_fraud_prob, n_samples)
    
    # Daily transaction count - high activity can be suspicious
    daily_count_base = np.random.poisson(3, n_samples)
    daily_count_risk = np.random.poisson(risk_scores * 10, n_samples)
    data['Daily_Transaction_Count'] = np.clip(daily_count_base + daily_count_risk, 1, 20)
    
    # 7-day average amount
    data['Avg_Transaction_Amount_7d'] = data['Transaction_Amount'] * np.random.uniform(0.7, 1.3, n_samples)
    
    # Failed transaction count - higher for risky users
    failed_base_prob = 0.8 * risk_scores
    data['Failed_Transaction_Count_7d'] = np.random.poisson(failed_base_prob * 5, n_samples)
    data['Failed_Transaction_Count_7d'] = np.clip(data['Failed_Transaction_Count_7d'], 0, 10)
    
    # Card types
    card_types = ['Visa', 'Mastercard', 'Amex']
    data['Card_Type'] = np.random.choice(card_types, n_samples, p=[0.5, 0.35, 0.15])
    
    # Card age - newer cards might be riskier
    data['Card_Age'] = np.random.exponential(100, n_samples).astype(int)
    data['Card_Age'] = np.clip(data['Card_Age'], 1, 2000)
    
    # Transaction distance - higher distances more suspicious
    distance_base = np.random.exponential(50, n_samples)
    distance_risk = risk_scores * np.random.exponential(500, n_samples)
    data['Transaction_Distance'] = distance_base + distance_risk
    data['Transaction_Distance'] = np.clip(data['Transaction_Distance'], 0.1, 8000)
    
    # Authentication methods
    auth_methods = ['Password', 'Biometric', 'OTP', '2FA']
    data['Authentication_Method'] = np.random.choice(auth_methods, n_samples, p=[0.4, 0.3, 0.2, 0.1])
    
    # Weekend flag
    data['Is_Weekend'] = np.random.binomial(1, 0.28, n_samples)
    
    # Generate realistic timestamps
    start_date = pd.Timestamp('2023-01-01')
    end_date = pd.Timestamp('2023-12-31')
    timestamps = pd.date_range(start_date, end_date, periods=n_samples)
    data['Timestamp'] = np.random.choice(timestamps, n_samples)
    
    df = pd.read_csv('fraud_dataset.csv')
    
    # Generate fraud labels with complex realistic patterns
    fraud_probability = (
        0.05 +  # Base fraud rate
        0.35 * (df['Risk_Score'] > 0.7) +  # High risk score
        0.25 * (df['Transaction_Amount'] > df['Transaction_Amount'].quantile(0.95)) +  # Very high amounts
        0.3 * df['IP_Address_Flag'] +  # Suspicious IP
        0.4 * df['Previous_Fraudulent_Activity'] +  # Previous fraud history
        0.2 * (df['Failed_Transaction_Count_7d'] > 3) +  # Multiple recent failures
        0.15 * (df['Transaction_Distance'] > df['Transaction_Distance'].quantile(0.9)) +  # Far transactions
        0.1 * (df['Daily_Transaction_Count'] > 10) +  # High daily activity
        0.08 * (df['Transaction_Type'] == 'Online') +  # Online transactions riskier
        0.06 * (df['Authentication_Method'] == 'Password') +  # Weak auth
        0.05 * df['Is_Weekend'] +  # Weekend transactions
        0.1 * (df['Card_Age'] < 30)  # New cards
    )
    
    # Add some noise and cap probability
    fraud_probability += np.random.normal(0, 0.05, n_samples)
    fraud_probability = np.clip(fraud_probability, 0.01, 0.85)
    
    # Generate labels
    df['Fraud_Label'] = np.random.binomial(1, fraud_probability, n_samples)
    
    print(f"✅ Data generated successfully!")
    print(f"📊 Fraud rate: {df['Fraud_Label'].mean():.1%}")
    print(f"📈 High-risk transactions: {(df['Risk_Score'] > 0.7).mean():.1%}")
    
    return df

# ============================================================================
# 2. ADVANCED FEATURE ENGINEERING
# ============================================================================

def advanced_feature_engineering(df):
    """Enhanced feature engineering with domain knowledge"""
    
    print("\n🔧 ADVANCED FEATURE ENGINEERING")
    print("=" * 45)
    
    df_processed = df.copy()
    
    # 1. Time-based features (enhanced)
    print("⏰ Creating advanced time features...")
    df_processed['Timestamp'] = pd.to_datetime(df_processed['Timestamp'])
    df_processed['Hour'] = df_processed['Timestamp'].dt.hour
    df_processed['Day'] = df_processed['Timestamp'].dt.day
    df_processed['Month'] = df_processed['Timestamp'].dt.month
    df_processed['DayOfWeek'] = df_processed['Timestamp'].dt.dayofweek
    df_processed['Quarter'] = df_processed['Timestamp'].dt.quarter
    df_processed['IsWeekend'] = df_processed['Timestamp'].dt.weekday >= 5
    
    # Time-based risk categories
    df_processed['Is_Night'] = ((df_processed['Hour'] >= 23) | (df_processed['Hour'] <= 5)).astype(int)
    df_processed['Is_Business_Hours'] = ((df_processed['Hour'] >= 9) & (df_processed['Hour'] <= 17)).astype(int)
    df_processed['Is_Late_Evening'] = ((df_processed['Hour'] >= 20) & (df_processed['Hour'] < 23)).astype(int)
    df_processed['Is_Early_Morning'] = ((df_processed['Hour'] >= 6) & (df_processed['Hour'] < 9)).astype(int)
    
    # 2. Amount-based features (enhanced)
    print("💰 Creating sophisticated amount features...")
    df_processed['Amount_to_Balance_Ratio'] = df_processed['Transaction_Amount'] / (df_processed['Account_Balance'] + 1)
    df_processed['Amount_vs_7d_Avg'] = df_processed['Transaction_Amount'] / (df_processed['Avg_Transaction_Amount_7d'] + 1)
    
    # Amount percentiles and categories
    df_processed['Amount_Percentile'] = df_processed['Transaction_Amount'].rank(pct=True)
    df_processed['Is_Large_Transaction'] = (df_processed['Amount_Percentile'] > 0.95).astype(int)
    df_processed['Is_Small_Transaction'] = (df_processed['Amount_Percentile'] < 0.1).astype(int)
    df_processed['Is_Round_Amount'] = (df_processed['Transaction_Amount'] % 100 == 0).astype(int)
    
    # Log transformations for skewed features
    df_processed['Log_Transaction_Amount'] = np.log1p(df_processed['Transaction_Amount'])
    df_processed['Log_Account_Balance'] = np.log1p(df_processed['Account_Balance'])
    df_processed['Log_Transaction_Distance'] = np.log1p(df_processed['Transaction_Distance'])
    
    # 3. Risk-based features (enhanced)
    print("⚠️ Creating comprehensive risk features...")
    df_processed['Risk_Category'] = pd.cut(df_processed['Risk_Score'], 
                                         bins=[0, 0.2, 0.5, 0.8, 1.0], 
                                         labels=['Very_Low', 'Low', 'Medium', 'High'])
    df_processed['High_Risk_Flag'] = (df_processed['Risk_Score'] > 0.7).astype(int)
    df_processed['Very_High_Risk_Flag'] = (df_processed['Risk_Score'] > 0.9).astype(int)
    df_processed['Risk_Score_Squared'] = df_processed['Risk_Score'] ** 2
    
    # 4. Behavioral features (enhanced)
    print("🎯 Creating behavioral pattern features...")
    df_processed['High_Daily_Activity'] = (df_processed['Daily_Transaction_Count'] > df_processed['Daily_Transaction_Count'].quantile(0.8)).astype(int)
    df_processed['Very_High_Activity'] = (df_processed['Daily_Transaction_Count'] > df_processed['Daily_Transaction_Count'].quantile(0.95)).astype(int)
    df_processed['Multiple_Failures'] = (df_processed['Failed_Transaction_Count_7d'] > 1).astype(int)
    df_processed['Many_Failures'] = (df_processed['Failed_Transaction_Count_7d'] > 3).astype(int)
    df_processed['Far_Transaction'] = (df_processed['Transaction_Distance'] > df_processed['Transaction_Distance'].quantile(0.8)).astype(int)
    df_processed['Very_Far_Transaction'] = (df_processed['Transaction_Distance'] > df_processed['Transaction_Distance'].quantile(0.95)).astype(int)
    
    # Failure rate
    df_processed['Failure_Rate'] = df_processed['Failed_Transaction_Count_7d'] / (df_processed['Daily_Transaction_Count'] + 1)
    
    # 5. Card and authentication features
    print("💳 Creating card and authentication features...")
    df_processed['Is_New_Card'] = (df_processed['Card_Age'] < 30).astype(int)
    df_processed['Is_Old_Card'] = (df_processed['Card_Age'] > 365).astype(int)
    df_processed['Is_Weak_Auth'] = (df_processed['Authentication_Method'] == 'Password').astype(int)
    df_processed['Is_Strong_Auth'] = (df_processed['Authentication_Method'].isin(['Biometric', '2FA'])).astype(int)
    
    # 6. Interaction features
    print("🔗 Creating interaction features...")
    df_processed['Risk_Amount_Interaction'] = df_processed['Risk_Score'] * df_processed['Log_Transaction_Amount']
    df_processed['Risk_Distance_Interaction'] = df_processed['Risk_Score'] * df_processed['Log_Transaction_Distance']
    df_processed['Amount_Distance_Interaction'] = df_processed['Log_Transaction_Amount'] * df_processed['Log_Transaction_Distance']
    df_processed['Night_Weekend_Interaction'] = df_processed['Is_Night'] * df_processed['IsWeekend']
    df_processed['High_Risk_High_Amount'] = df_processed['High_Risk_Flag'] * df_processed['Is_Large_Transaction']
    
    # 7. Frequency encoding for categorical variables
    print("📊 Creating frequency encodings...")
    for col in ['Location', 'Merchant_Category', 'Device_Type']:
        freq_map = df_processed[col].value_counts(normalize=True).to_dict()
        df_processed[f'{col}_Frequency'] = df_processed[col].map(freq_map)
    
    # 8. Categorical encoding with label encoding
    print("🏷️ Encoding categorical variables...")
    categorical_cols = ['Transaction_Type', 'Device_Type', 'Location', 'Merchant_Category', 
                       'Card_Type', 'Authentication_Method', 'Risk_Category']
    
    le_dict = {}
    for col in categorical_cols:
        if col in df_processed.columns:
            le = LabelEncoder()
            df_processed[f'{col}_encoded'] = le.fit_transform(df_processed[col].astype(str))
            le_dict[col] = le
    
    # Drop original categorical columns and identifiers
    cols_to_drop = ['Transaction_ID', 'User_ID', 'Timestamp'] + \
                   [col for col in categorical_cols if col in df_processed.columns]
    df_processed = df_processed.drop(columns=cols_to_drop)
    
    print(f"✅ Advanced feature engineering complete!")
    print(f"📊 Final feature count: {df_processed.shape[1] - 1}")  # -1 for target
    print(f"🎯 Total samples: {df_processed.shape[0]:,}")
    
    return df_processed, le_dict

# ============================================================================
# 3. ENHANCED MODELING PIPELINE
# ============================================================================

class EnhancedFraudDetectionPipeline:
    """Enhanced fraud detection pipeline with better accuracy"""
    
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.results = {}
        self.best_model = None
        self.best_score = 0
        self.cv_results = {}
        
    def prepare_data_advanced(self, df_processed):
        """Advanced data preparation with multiple scaling options"""
        
        print("\n🎯 ADVANCED DATA PREPARATION")
        print("=" * 40)
        
        # Separate features and target
        X = df_processed.drop('Fraud_Label', axis=1)
        y = df_processed['Fraud_Label']
        
        print(f"Features shape: {X.shape}")
        print(f"Target distribution: {y.value_counts().to_dict()}")
        print(f"Fraud rate: {y.mean():.1%}")
        
        # Advanced train-test split with stratification
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Multiple scaling approaches
        # Standard scaling
        standard_scaler = StandardScaler()
        X_train_standard = standard_scaler.fit_transform(X_train)
        X_test_standard = standard_scaler.transform(X_test)
        
        # Robust scaling (less sensitive to outliers)
        robust_scaler = RobustScaler()
        X_train_robust = robust_scaler.fit_transform(X_train)
        X_test_robust = robust_scaler.transform(X_test)
        
        self.scalers['standard'] = standard_scaler
        self.scalers['robust'] = robust_scaler
        
        return (X_train, X_test, y_train, y_test, 
                X_train_standard, X_test_standard,
                X_train_robust, X_test_robust)
    
    def advanced_class_balancing(self, X_train_scaled, y_train):
        """Advanced class balancing techniques"""
        
        print("\n⚖️ ADVANCED CLASS BALANCING")
        print("=" * 35)
        
        original_distribution = Counter(y_train)
        print(f"Original distribution: {dict(original_distribution)}")
        
        balanced_datasets = {}
        
        if HAS_IMBLEARN:
            # SMOTE variants
            techniques = {
                'SMOTE': SMOTE(random_state=42, k_neighbors=3),
                'BorderlineSMOTE': BorderlineSMOTE(random_state=42, k_neighbors=3),
                'ADASYN': ADASYN(random_state=42, n_neighbors=3),
                'SMOTETomek': SMOTETomek(random_state=42)
            }
            
            for name, technique in techniques.items():
                try:
                    X_balanced, y_balanced = technique.fit_resample(X_train_scaled, y_train)
                    balanced_datasets[name] = (X_balanced, y_balanced)
                    print(f"✅ {name}: {X_balanced.shape[0]:,} samples")
                except Exception as e:
                    print(f"⚠️ {name} failed: {str(e)}")
        
        # Fallback: Enhanced manual balancing
        if not balanced_datasets:
            print("Using enhanced manual balancing...")
            
            # Separate classes
            minority_class = y_train.value_counts().idxmin()
            majority_class = y_train.value_counts().idxmax()
            
            minority_indices = np.where(y_train == minority_class)[0]
            majority_indices = np.where(y_train == majority_class)[0]
            
            # Create balanced dataset with some noise
            n_minority = len(minority_indices)
            n_majority = len(majority_indices)
            
            # Oversample minority class with noise
            minority_oversampled = []
            for _ in range(n_majority - n_minority):
                idx = np.random.choice(minority_indices)
                sample = X_train_scaled[idx].copy()
                # Add small amount of noise
                noise = np.random.normal(0, 0.01, sample.shape)
                minority_oversampled.append(sample + noise)
            
            if minority_oversampled:
                X_minority_over = np.vstack([X_train_scaled[minority_indices]] + minority_oversampled)
                y_minority_over = np.full(len(X_minority_over), minority_class)
                
                X_balanced = np.vstack([X_train_scaled[majority_indices], X_minority_over])
                y_balanced = np.concatenate([y_train.iloc[majority_indices], y_minority_over])
                
                balanced_datasets['Manual'] = (X_balanced, y_balanced)
                print(f"✅ Manual balancing: {X_balanced.shape[0]:,} samples")
        
        # Return the best balanced dataset (prefer SMOTE variants)
        if 'SMOTETomek' in balanced_datasets:
            return balanced_datasets['SMOTETomek']
        elif 'SMOTE' in balanced_datasets:
            return balanced_datasets['SMOTE']
        elif balanced_datasets:
            return list(balanced_datasets.values())[0]
        else:
            print("⚠️ No balancing applied, using original data")
            return X_train_scaled, y_train
    
    def build_enhanced_models(self, X_train_balanced, y_train_balanced, X_train_scaled):
        """Build enhanced models with hyperparameter tuning"""
        
        print("\n🤖 BUILDING ENHANCED MODELS")
        print("=" * 35)
        
        # Enhanced model configurations
        models_config = {
            'Logistic Regression': {
                'model': LogisticRegression(random_state=42, max_iter=2000),
                'params': {
                    'C': [0.1, 1, 10, 100],
                    'penalty': ['l1', 'l2'],
                    'solver': ['liblinear', 'saga']
                }
            },
            'Random Forest': {
                'model': RandomForestClassifier(random_state=42, n_jobs=-1),
                'params': {
                    'n_estimators': [100, 200, 300],
                    'max_depth': [10, 20, None],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4]
                }
            },
            'Gradient Boosting': {
                'model': GradientBoostingClassifier(random_state=42),
                'params': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1, 0.2],
                    'max_depth': [3, 5, 7]
                }
            }
        }
        
        # Add XGBoost if available
        if HAS_XGBOOST:
            models_config['XGBoost'] = {
                'model': xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
                'params': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1, 0.2],
                    'max_depth': [3, 5, 7],
                    'subsample': [0.8, 0.9, 1.0]
                }
            }
        
        # Add CatBoost if available
        if HAS_CATBOOST:
            models_config['CatBoost'] = {
                'model': CatBoostClassifier(random_state=42, verbose=False),
                'params': {
                    'iterations': [100, 200, 300],
                    'learning_rate': [0.05, 0.1, 0.2],
                    'depth': [4, 6, 8]
                }
            }
        
        # Train models with hyperparameter tuning
        for name, config in models_config.items():
            print(f"Training {name} with hyperparameter tuning...")
            try:
                # Use GridSearchCV for hyperparameter tuning
                grid_search = GridSearchCV(
                    config['model'], 
                    config['params'],
                    cv=3,  # 3-fold CV for speed
                    scoring='f1',
                    n_jobs=-1,
                    verbose=0
                )
                
                grid_search.fit(X_train_balanced, y_train_balanced)
                
                self.models[name] = grid_search.best_estimator_
                self.cv_results[name] = {
                    'best_score': grid_search.best_score_,
                    'best_params': grid_search.best_params_
                }
                
                print(f"✅ {name} - Best CV F1: {grid_search.best_score_:.4f}")
                
            except Exception as e:
                print(f"⚠️ Error training {name}: {e}")
                # Fallback to default parameters
                try:
                    config['model'].fit(X_train_balanced, y_train_balanced)
                    self.models[name] = config['model']
                    print(f"✅ {name} - Using default parameters")
                except Exception as e2:
                    print(f"❌ {name} failed completely: {e2}")
        
        # Add ensemble methods
        if len(self.models) >= 2:
            print("Creating ensemble models...")
            # Voting classifier would go here, but skipping for simplicity
        
        return X_train_balanced, y_train_balanced
    
    def comprehensive_evaluation(self, X_test, y_test, X_test_scaled):
        """Comprehensive model evaluation with multiple metrics"""
        
        print("\n📊 COMPREHENSIVE MODEL EVALUATION")
        print("=" * 40)
        
        results = []
        
        for name, model in self.models.items():
            print(f"\nEvaluating {name}...")
            
            try:
                # Make predictions
                y_pred = model.predict(X_test_scaled)
                
                if hasattr(model, 'predict_proba'):
                    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
                else:
                    # For models without predict_proba
                    y_pred_proba = model.decision_function(X_test_scaled)
                    # Normalize to [0,1]
                    y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())
                
                # Calculate comprehensive metrics
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                roc_auc = roc_auc_score(y_test, y_pred_proba)
                avg_precision = average_precision_score(y_test, y_pred_proba)
                
                # Additional metrics
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
                specificity = tn / (tn + fp)
                balanced_accuracy = (recall + specificity) / 2
                
                results.append({
                    'Model': name,
                    'Precision': precision,
                    'Recall': recall,
                    'F1-Score': f1,
                    'ROC-AUC': roc_auc,
                    'Avg-Precision': avg_precision,
                    'Specificity': specificity,
                    'Balanced-Acc': balanced_accuracy,
                    'CV-F1': self.cv_results.get(name, {}).get('best_score', 0)
                })
                
                # Store results
                self.results[name] = {
                    'y_pred': y_pred,
                    'y_pred_proba': y_pred_proba,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'roc_auc': roc_auc,
                    'avg_precision': avg_precision,
                    'specificity': specificity,
                    'balanced_accuracy': balanced_accuracy
                }
                
                # Update best model based on F1 score
                if f1 > self.best_score:
                    self.best_score = f1
                    self.best_model = name
                    
            except Exception as e:
                print(f"⚠️ Error evaluating {name}: {e}")
        

⚠️ imbalanced-learn not available. Using alternative class balancing methods.
🚀 Enhanced Fraud Detection System Initialized
